In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub

In [2]:
tf.config.experimental.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [9]:
IMAGE_SHAPE = (224, 224)

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
#PIL.Image.open('C:/Users/ys726/Desktop/DL/transfer_learning/goldfish.jpg')
gold_fish = PIL.Image.open('./datasets/goldfish.jpg').resize(IMAGE_SHAPE)
gold_fish

In [ ]:
gold_fish = np.array(gold_fish)/225
gold_fish.shape

In [ ]:
gold_fish

In [ ]:
'''We are adding one more dimenstion. the reason we are doing this because 
   when we do prediction so prediction accept multiple image as an input'''
gold_fish[np.newaxis, ...].shape

In [ ]:
result = classifier.predict(gold_fish[np.newaxis, ...])
result.shape

In [ ]:
result

In [ ]:
predicted_label_index = np.argmax(result)
predicted_label_index

In [ ]:
image_labels = []
with open("./datasets/ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()

image_labels[:5]

In [ ]:
image_labels[predicted_label_index]

## Use this pretrained model for our flowers dataset.

In [3]:
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" 
data_dir = tf.keras.utils.get_file('flower_photos', origin=url, cache_dir='.', untar=True)

228813984/228813984 [==============================] - 39s 0us/step


In [4]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('datasets/flower_photos')

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

3670

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
PIL.Image.open(roses[6])

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
tulips[:5]

In [ ]:
PIL.Image.open(tulips[0])

In [6]:
flowers_image_dict = {
    'roses' : list(data_dir.glob('roses/*')),
    'tulips' : list(data_dir.glob('tulips/*')),
    'dandelion' : list(data_dir.glob('dandelion/*')),
    'sunflowers' : list(data_dir.glob('sunflowers/*')),
    'daisy' : list(data_dir.glob('daisy/*')),
}

In [7]:
flowers_labels_dict = {
    'roses':0,
    'daisy':1,
    'dandelion':2,
    'sunflowers':3,
    'tulips':4
}

In [10]:
# Creating the dataset X contains flower images (3D numpy array) and y contains flower name (flower number)
X, y = [], []
for flower_name, images in flowers_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_image = cv2.resize(img,IMAGE_SHAPE)
        X.append(resized_image)
        y.append(flowers_labels_dict[flower_name])


In [11]:
# Converting list into numpy array
X = np.array(X)
y = np.array(y)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [13]:
print(len(X_train))
print(len(X_test))

2752
918


In [14]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [15]:
print(X_train_scaled.shape)
f"input layer size = {224*224*3}"

(2752, 224, 224, 3)


'input layer size = 150528'

In [ ]:
plt.axis('off')
plt.imshow(X[1])

In [ ]:
plt.axis('off')
plt.imshow(X[2])

In [ ]:
# Lry to use our classifier to pridict this model & classifier is the pretrained model
predicted = classifier.predict(np.array([X[0], X[1], X[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

In [ ]:
image_labels[795], image_labels[880],image_labels[795]

<b> So here we can not use readymate model.


In [16]:
X.shape

(3670, 224, 224, 3)

In [19]:
y.shape

(3670,)

In [17]:
# Now I am going to retrain this model.

'''This will give us the same model as 
    previous one except the last layer'''

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False 
    ) # trainable=False means --> Don't train all those layes have fixed weights


In [18]:
num_of_flowers = 5 # (or output shape)

model = tf.keras.Sequential([
    #Putting that readymate model
    pretrained_model_without_top_layer,

    # Dense Network
    tf.keras.layers.Dense(num_of_flowers, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy']
#               )

# model.fit(X_train_scaled, y_train, epochs=1, batch_size=1000)

In [ ]:
# Creating a function for applying the model
def get_model():
    
    model = tf.keras.Sequential([
    #Putting that readymate model
    pretrained_model_without_top_layer,

    # Dense Network
    tf.keras.layers.Dense(num_of_flowers, activation="softmax")
        ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )
    
    return model

In [ ]:
# Running on CPU
import time
start_time = time.time()

with tf.device('/CPU:0'):
    cpu_model = get_model()
    cpu_model.fit(X_train_scaled, y_train, epochs=5)

end_time = time.time()
print("CPU Execution time:", end_time - start_time)

In [ ]:
# # Running on GPU
# start_time = time.time()

# with tf.device('/GPU:0'):
#     gpu_model = get_model()
#     gpu_model.fit(X_train_scaled, y_train, epochs=5)

# end_time = time.time()
# print("GPU Execution time:", end_time - start_time)